# This file was intended to load [Tensorflow Model Garden](https://www.tensorflow.org/guide/model_garden) Models for detection, and to verify GPU's are correctly interacting with Tensorflow
## This turned out to be more of a pain and an existing installation was used instead
### Consider this a 'just in case' file

[Model Garden Git](https://github.com/tensorflow/models)

In [2]:
import os
import pathlib
import cv2 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import sys
import tensorflow as tf
# import object_detection.utils as utils

from tensorflow import keras
from tensorflow.python.client import device_lib

In [5]:
!nvidia-smi
!nvcc -V
sys.path.append('..')
print('Python Version: ', sys.version.split()[0])
print('Tensorflow/Keras Version: ', tf.__version__)
print('Devices:')
print(device_lib.list_local_devices()) # check active GPUs

Fri Jan 13 22:34:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:17:00.0 Off |                  N/A |
|  0%   34C    P8     7W / 260W |     10MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:18:00.0 Off |                  N/A |
|  0%   

2023-01-13 22:34:35.627122: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-13 22:34:43.107333: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:0 with 9629 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5
2023-01-13 22:34:43.116742: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:1 with 9629 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:18:00.0, compute capability: 7.5
2023-01-13 22:34:43.117845: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:2 with 9043 MB memory:  -> device: 2, name: NVIDIA GeForce RTX 2080 Ti

In [7]:
if "models" in pathlib.Path.cwd().parts:
    while "models" in pathlib.Path.cwd().parts:
        os.chdir('..')
elif not pathlib.Path('models').exists():
    !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3657, done.
remote: Counting objects: 100% (3657/3657), done.
remote: Compressing objects: 100% (3053/3053), done.
remote: Total 3657 (delta 969), reused 1570 (delta 552), pack-reused 0
Receiving objects: 100% (3657/3657), 47.39 MiB | 6.23 MiB/s, done.
Resolving deltas: 100% (969/969), done.


In [ ]:
%%bash
ls
cd models/
ls
cd research/
protoc object_detection/protos/*.proto --python_out=.

pip install .

In [ ]:
utils_ops.tf = tf.compat.v1
tf.gfile = tf.io.gfile

def load_model(model_name):
    base_url = 'http://download.tensorflow.org/models/object_detection/'
    model_file = model_name + '.tar.gz'
    model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

    model_dir = pathlib.Path(model_dir)/"saved_model"
    model = tf.saved_model.load(str(model_dir))

    return model

model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
detection_model = load_model(model_name)